In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

import time
import random
import re
from itertools import product

import pandas as pd

In [2]:
def search_matrix(rooms=None, floor=None, is_apartment=None, not_last_floor=None):
    '''
    функция служит для создания матрицы поиска, которая будет использоваться 
    парсером. Таким образом можно существенно увеличить количество объявлений.
    
    Принимает на вход:
    rooms - целое, список - количество комнат
    floor - целое, список - этаж
    is_apartment - boolean - является ли объект апартементами
    last_floor - boolean - является ли этаж последним
    
    Возвращает:
    список кортежей с заданными параметрами. 
    Индексация кортежей 
    [0] - количество комнат
    [1] - этаж
    [2] - апартаменты True / False
    [3] - последний этаж True / False
    '''
    if not rooms:
        rooms = [i for i in range(1,10)]
    elif type(rooms) == list:
        pass
    else:
        rooms = [rooms]
    
    if not floor:
        floor = [i for i in range(1,50)]
    elif type(floor) == list:
        pass
    else:
        floor = [floor]
    
    if not is_apartment:
        if is_apartment == False:
            is_apartment = [is_apartment]
        else:
            is_apartment = [True, False]
    else:
        is_apartment = [is_apartment]
        
    if not not_last_floor:
        if not_last_floor == False:
            not_last_floor = [not_last_floor]
        else:
            not_last_floor = [True, False]
    else:
        not_last_floor = [not_last_floor]
        
    return list(product(rooms, floor, is_apartment, not_last_floor))

In [3]:
def search_url(params):
    base_link = f'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2'
    
    rooms = params[0]
    floor = params[1]
    is_apartment = params[2]
    not_last_floor = params[3]
    search_url = base_link + \
    f'&floornl={not_last_floor*1}' + \
    f'&maxfloor={floor}&minfloor={floor}' + \
    '&offer_type=flat'
    
    if is_apartment:
        search_url = search_url[:28] + 'apartment=1&' + search_url[28:]
    else:
        search_url = search_url + '&only_flat=1'
    
    search_url += f'&p=1&region=1&room{rooms}=1'
    return search_url

In [4]:
def parse_cian_with_defined_params(rooms=1, floor=1, is_apartment=None, not_last_floor=None):
    '''
    Ходит по страницам циана и собирает нужную информацию
    
    Если не задавать параметры, то разделяет самостоятельно объявления по параметрам комнатности и этажности
    При этом для каждой ячейки этой матрицы собирает максимум 1512 объявлений (ограничение циана)
    
    Возвращает - словарь с объявлениями. Ключ - id объявления.
    '''
    
    driver = webdriver.Chrome()
    #задержки можно убрать на свой страх и риск
#     time.sleep(2)
    result = {}
    
    grid = search_matrix(rooms, floor, is_apartment, not_last_floor)
    
    print(f'Всего запросов: {len(grid)}')
    for count, params in enumerate(grid):
        page = 1
        url = search_url(params)
        driver.get(url)

        try:
            while True:
                #задержки можно убрать на свой страх и риск
#                 time.sleep(1)
                soup = BeautifulSoup(driver.page_source,'lxml')
                print(f'\rОбрабатывается запрос: {count+1}, Уникальных записей в словаре: {len(result)}', end='')
                #тут карточки всех объектов без доп.предложений
                offers_summary = soup.find('div','_93444fe79c--wrapper--W0WqH')
                #обход блока циана
                if offers_summary is None:
                    time.sleep(10)
                    driver.get(url)
                    continue
                for i in offers_summary.find_all('div','_93444fe79c--card--ibP42'):
                    # i - карточка отдельного объекта
                    
                    link = i.find('a', '_93444fe79c--link--eoxce')['href']
                    ad_id = link.split('/')[-2]
                    result[ad_id] = {}
                    result[ad_id]['link'] = link
                    
                    result[ad_id]['rooms'] = params[0]
                    result[ad_id]['floor'] = params[1]
                    result[ad_id]['is_apartment'] = params[2]
                    result[ad_id]['not_last_floor'] = params[3]
                
                    title = i.find('span',{'data-mark':'OfferTitle'})
                    result[ad_id]['title'] = title.text
        
                    try:
                        subtitle = i.find('span',{'data-mark':'OfferSubtitle'})
                        result[ad_id]['subtitle'] = subtitle.text
                    except:
                        pass
        
                    try:
                        deadline = i.find('span', {'data-mark':'Deadline'})
                        result[ad_id]['deadline'] = deadline.text
                    except:
                        pass
        
                    try:
                        metro = i.find('a','_93444fe79c--link--BwwJO')
                        result[ad_id]['metro'] = metro.text
                        metro_remote = i.find('div','_93444fe79c--remoteness--q8IXp')
                        result[ad_id]['metro_remote'] = metro_remote.text
        
                    except:
                        pass
        
                    geo = i.find('div','_93444fe79c--labels--L8WyJ')
                    result[ad_id]['geo'] = geo.text
        
                    main_price = i.find('span',{'data-mark':'MainPrice'})
                    result[ad_id]['main_price'] = int(re.sub(' ', '',main_price.text[:-2]))
                    if main_price.text[-1] == '₽':
                        currency = 'rouble'
                    else:
                        currency = 'other'
                    result[ad_id]['currency'] = currency
        
                    price_for_sq_m = i.find('p',{'data-mark':'PriceInfo'})
                    result[ad_id]['price_for_sq_m'] = int(re.sub(' ', '',price_for_sq_m.text[:-5]))
        
                
                #проверяем наличие заблокированных кнопок
                if soup.find_all('button',{'data-name':'PaginationButton'}):
                    #заблокированные кнопки есть, если последняя это "Дальше", то начинаем поиск по новым параметрам
                    if soup.find_all('button',{'data-name':'PaginationButton'})[-1].text == 'Дальше':
                        break
                    #в ином случае заблоченные кнопки есть, но "Дальше" активна, переворачиваем страницу
                    else:
                        page += 1
                        url = re.sub(f'p={page-1}', f'p={page}', url)
                        driver.get(url)
                        #задержки можно убрать на свой страх и риск
#                         time.sleep(1)
                        
                #если кнопок нет, то начинаем поиск по новым параметрам
                else:
                    break
        #ошибка или ручное прерывание            
        except Exception as e:
            print(e)
            print(f'\nПоиск завершился с ошибкой, Параметры: {params}, Страница {url}')
            return result, soup
    #основной цикл закончен, выводим результат
    print('\nПоиск по сетке завершен')
    driver.quit()
    return result, soup    
            

In [5]:
time.sleep(1)

In [6]:
%%time
parsed_data, last_soup = parse_cian_with_defined_params(rooms=[1,8], floor=[10,20], is_apartment=None, not_last_floor=True)

Всего запросов: 8
Обрабатывается запрос: 8, Уникальных записей в словаре: 1384
Поиск по сетке завершен
CPU times: total: 13.7 s
Wall time: 3min 59s


In [7]:
df = pd.DataFrame(parsed_data).T
df

,link,rooms,floor,is_apartment,not_last_floor,title,subtitle,deadline,metro,metro_remote,geo,main_price,currency,price_for_sq_m
286010735,https://www.cian.ru/sale/flat/286010735/,1,10,True,True,"1-комн. апарт., 42,7 м², 10/13 этаж",Сдача корпуса 3 кв. 2024,сдача ГК: 3 кв. 2024 года,Тульская,8 минут пешком,"Москва, ЮАО, р-н Даниловский, м. Тульская, Под...",28079459,rouble,657599
286139851,https://www.cian.ru/sale/flat/286139851/,1,10,True,True,"1-комн. апарт., 17,2 м², 10/22 этаж",NaN,NaN,Юго-Западная,16 минут пешком,"Москва, ЗАО, р-н Тропарево-Никулино, м. Юго-За...",6475800,rouble,376500
281557867,https://www.cian.ru/sale/flat/281557867/,1,10,True,True,"1-комн. апарт., 45,65 м², 10/14 этаж",Сдан,дом сдан,Тёплый Стан,10 минут пешком,"Москва, ЮЗАО, р-н Ясенево, м. Тёплый Стан, Нов...",13447000,rouble,294567
288276062,https://www.cian.ru/sale/flat/288276062/,1,10,True,True,"Апартаменты с ремонтом, мебелью!","1-комн. апарт., 17,2 м², 10/22 этаж",NaN,Тропарёво,12 минут пешком,"Москва, ЗАО, р-н Тропарево-Никулино, м. Тропар...",6475800,rouble,376500
285674285,https://www.cian.ru/sale/flat/285674285/,1,10,True,True,"1-комн. апарт., 68,9 м², 10/17 этаж",Сдача корпуса 4 кв. 2024,сдача ГК: 4 кв. 2024 года,Филёвский парк,11 минут пешком,"Москва, ЗАО, р-н Филевский парк, м. Филёвский ...",36785710,rouble,533900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284065280,https://www.cian.ru/sale/flat/284065280/,8,10,False,True,"1/2 квартиры, 50,4 м², 10/12 этаж",NaN,NaN,Улица Академика Янгеля,19 минут пешком,"Москва, ЮАО, р-н Чертаново Южное, м. Улица Ака...",5000000,rouble,99206
279532243,https://www.cian.ru/sale/flat/279532243/,8,10,False,True,"2/3 квартиры, 74 м², 10/17 этаж",NaN,NaN,Ясенево,15 минут пешком,"Москва, ЮЗАО, р-н Ясенево, м. Ясенево, Голубин...",5800000,rouble,78378
267884876,https://www.cian.ru/sale/flat/267884876/,8,10,False,True,"1/2 квартиры, 75,8 м², 10/17 этаж",NaN,NaN,Рассказовка,5 минут на транспорте,"Москва, НАО (Новомосковский), м. Рассказовка, ...",7000000,rouble,92348
286689800,https://www.cian.ru/sale/flat/286689800/,8,10,False,True,"1/2 квартиры, 65,3 м², 10/12 этаж",NaN,NaN,Отрадное,5 минут на транспорте,"Москва, СВАО, р-н Южное Медведково, м. Отрадно...",7500000,rouble,114855


In [8]:
%%time
parse_all_data, last_soup = parse_cian_with_defined_params(None, None, None, None)

Всего запросов: 1764
Обрабатывается запрос: 1764, Уникальных записей в словаре: 99512
Поиск по сетке завершен
CPU times: total: 18min 45s
Wall time: 4h 17min 50s


In [9]:
df = pd.DataFrame(parse_all_data).T
df

,link,rooms,floor,is_apartment,not_last_floor,title,subtitle,deadline,metro,metro_remote,geo,main_price,currency,price_for_sq_m
290090154,https://www.cian.ru/sale/flat/290090154/,1,1,True,True,"1-комн. апарт., 40,05 м², 1/5 этаж",Сдача корпуса 4 кв. 2024,сдача ГК: 3 кв. 2024 года,Площадь Революции,2 минуты пешком,"Москва, ЦАО, р-н Тверской, м. Площадь Революци...",82040000,rouble,2048439
283849683,https://www.cian.ru/sale/flat/283849683/,1,1,True,True,"Новая студия 12,5 кв.м","1-комн. апарт., 12,5 м², 1/5 этаж",NaN,Волжская,5 минут пешком,"Москва, ЮВАО, р-н Текстильщики, м. Волжская, у...",4000000,rouble,320000
270246372,https://www.cian.ru/sale/flat/270246372/,1,1,True,True,"1-комн. апарт., 37,3 м², 1/8 этаж",Секция 1 • Сдача корпуса 2 кв. 2024,сдача ГК: 2 кв. 2024 года,Петровский Парк,5 минут пешком,"Москва, САО, р-н Савеловский, м. Петровский Па...",14346459,rouble,384624
288719837,https://www.cian.ru/sale/flat/288719837/,1,1,True,True,Новая студия на Западе Москвы,"1-комн. апарт., 14,4 м², 1/17 этаж",NaN,Кунцевская,5 минут на транспорте,"Москва, ЗАО, р-н Можайский, м. Кунцевская, Дор...",4100000,rouble,284722
287829364,https://www.cian.ru/sale/flat/287829364/,1,1,True,True,"1-комн. апарт., 39 м², 1/8 этаж",Сдача корпуса 2 кв. 2024,сдача ГК: 2 кв. 2024 года,Петровский Парк,5 минут пешком,"Москва, САО, р-н Савеловский, м. Петровский Па...",16125603,rouble,413477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275407378,https://www.cian.ru/sale/flat/275407378/,9,48,False,True,"Студия, 25,6 м², 48/52 этаж",Секция 1 • Сдача корпуса 2 кв. 2026,сдача ГК: 2 кв. 2026 года,Москворечье,18 минут пешком,"Москва, ЮАО, р-н Москворечье-Сабурово, м. Моск...",11649280,rouble,455050
288222683,https://www.cian.ru/sale/flat/288222683/,9,48,False,True,"Студия, 30,74 м², 48/54 этаж",Сдача корпуса 2 кв. 2025,сдача ГК: 2 кв. 2025 года,Дмитровская,5 минут пешком,"Москва, САО, р-н Савеловский, м. Дмитровская, ...",15870613,rouble,516285
288017653,https://www.cian.ru/sale/flat/288017653/,9,49,True,True,"Апартаменты-студия, 27,2 м², 49/85 этаж",NaN,NaN,Деловой центр,3 минуты пешком,"Москва, ЦАО, р-н Пресненский, м. Деловой центр...",25515000,rouble,938051
289981727,https://www.cian.ru/sale/flat/289981727/,9,49,True,True,"Апартаменты-студия, 16,7 м², 49/85 этаж",NaN,NaN,Деловой центр,3 минуты пешком,"Москва, ЦАО, р-н Пресненский, м. Деловой центр...",20200000,rouble,1209581


In [11]:
df.to_json('cian_19_07.json')